In [8]:
%pip install beautifulsoup4 requests

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [beautifulsoup4]m [beautifulsoup4]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests

base_url = "https://www.asx.com.au"
# todayAnns.do (cross section)
# announcements.do
year = 2026
sym = 'SOL'

response = requests.get(f"{base_url}/asx/v2/statistics/announcements.do", params={
  'by': 'asxCode',
  'asxCode': sym,
  'timeframe': 'Y',
  'year': year
})


In [39]:
from bs4 import BeautifulSoup
from datetime import datetime, timezone
from zoneinfo import ZoneInfo
from urllib.parse import urlparse, parse_qs

soup = BeautifulSoup(response.content, 'html.parser')

def extract_cells(date, price, link):
  return [
    datetime.strptime(date.get_text(' ', strip=True), r'%d/%m/%Y %I:%M %p').replace(tzinfo=ZoneInfo("Australia/Sydney")).astimezone(timezone.utc),
    price.img is not None,
    link.a.get_text(strip=True).split('\n')[0],
    parse_qs(urlparse(link.a['href']).query)['idsId'][0] #/asx/v2/statistics/displayAnnouncement.do?display=pdf&idsId=1234
  ]

for row in soup.select('announcement_data tbody > tr'):
  print(extract_cells(*row.find_all('td')))

[datetime.datetime(2026, 1, 23, 5, 55, tzinfo=datetime.timezone.utc), False, 'Change in substantial holding for AIS4', '03052509']
[datetime.datetime(2026, 1, 6, 6, 25, tzinfo=datetime.timezone.utc), False, 'Change in substantial holding for AIS4', '03047601']
